In [1]:
import unittest
from ddt import ddt, data, unpack
from itertools import product

import sys
sys.path.append('../Algorithms/')
import ActionInterpretation as targetCode
sys.path.append('../Environments/')
from ColoredGridWorld import SetUpInferenceSpace as sup

In [52]:
@ddt
class TestActionInterpretation(unittest.TestCase):
    def setUp(self): 
        dimensions = (3,3)
        self.goals = [(2,1)]
        actions = [(-1,0),(0,1),(0,-1),(1,0)]
        oneColourStates1 = {(0,0): 'white', (0,1): 'white', (0,2):'white', (1,0): 'white', (1,1): 'orange', (1,2):'white', (2,0): 'white', (2,1): 'yellow', (2,2):'white'}
        oneColourStates2 = {(0,0): 'white', (0,1): 'white', (0,2):'white', (1,0): 'orange', (1,1): 'white', (1,2):'orange', (2,0): 'white', (2,1): 'yellow', (2,2):'white'}
        twoColourStates = {(0,0): 'white', (0,1): 'white', (0,2):'white', (1,0): 'white', (1,1): 'orange', (1,2):'blue', (2,0): 'white', (2,1): 'yellow', (2,2):'white'}
        self.traj1 = [(0,1), (0,1), (0,2), (1,0), (1,2), (1,0), (2,2), (0,-1), (2,1)]
        self.traj2 = [(0,1), (1,0), (1,1), (1,0), (2,1)]
        traj3 = [(0,1), (0,-1), (0,0), (1,0), (1,0), (1,0), (2,0), (0,1), (2,1)]
        traj4 = [(0,1), (0,1), (0,2)]
        convergenceTolerance = 10e-7
        gamma = 0.94
        alpha = 20
        eps = 0.01
        hyperparameters = (convergenceTolerance, gamma, alpha, eps)
        oneColourUtilitySpace = [{'white': 0, 'yellow': 10, 'orange': 0}, {'white': 0, 'yellow': 10, 'orange': -2}]
        twoColourUtilitySpace = [ {'white': 0, 'yellow': 10, 'orange': 0, 'blue':0}, {'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},{'white': 0, 'yellow': 10, 'orange': 0, 'blue':-2}, {'white': 0, 'yellow': 10, 'orange': -2, 'blue':-2}]
        transitionSpace = [True]
        oneColourWorlds = sup.buildWorldSpace(oneColourUtilitySpace, transitionSpace)
        twoColourWorlds = sup.buildWorldSpace(twoColourUtilitySpace, transitionSpace)
        oneColourEnvSpace = [(world, goal) for world, goal in product(oneColourWorlds, self.goals)]
        twoColourEnvSpace = [(world, goal) for world, goal in product(twoColourWorlds, self.goals)]
        oneColourEnvPolicySpace1 = sup.buildEnvPolicySpace(dimensions, oneColourStates1, actions, oneColourEnvSpace, hyperparameters)
        oneColourEnvPolicySpace2 = sup.buildEnvPolicySpace(dimensions, oneColourStates2, actions, oneColourEnvSpace, hyperparameters)
        twoColourEnvPolicySpace = sup.buildEnvPolicySpace(dimensions, twoColourStates, actions, twoColourEnvSpace, hyperparameters)
        self.oneColourBeliefVector1 = targetCode.ActionInterpretation(oneColourEnvPolicySpace1)
        self.oneColourBeliefVector2 = targetCode.ActionInterpretation(oneColourEnvPolicySpace2)
        self.twoColourBeliefVector = targetCode.ActionInterpretation(twoColourEnvPolicySpace) 
        self.oneColourUniformPriors = {(env): (1/len(oneColourEnvSpace)) for env in oneColourEnvSpace}
        self.nonUniformPriors = {oneColourEnvSpace[0]:0.25, oneColourEnvSpace[1]:0.75}
        self.twoColourUniformPriors = {(env): (1/len(twoColourEnvSpace)) for env in twoColourEnvSpace} 

    # Action Interpretion #######################################################################################################
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0},True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2}, True)))
    @unpack
    def test_actionInterpretationPosterior_equivalentEnvBeliefs_oneColourStateSpace2(self, worldA, worldB):
        #set up the correct combination of worlds trajectories and environments
        posteriorProbabilityOfWorldA = self.oneColourBeliefVector2(self.traj2, self.oneColourUniformPriors, worldA, self.goals[0], False)
        posteriorProbabilityOfWorldB = self.oneColourBeliefVector2(self.traj2, self.oneColourUniformPriors, worldB, self.goals[0], False)
        self.assertAlmostEqual(posteriorProbabilityOfWorldA, posteriorProbabilityOfWorldB)
        
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0},True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2}, True)))
    @unpack
    def test_actionInterpretationPosterior_equivalentEnvBeliefs_oneColourStateSpace2_nonUniformPrior(self, worldA, worldB):
        #set up the correct combination of worlds trajectories and environments
        posteriorProbabilityOfWorldA = self.oneColourBeliefVector2(self.traj2, self.nonUniformPriors, worldA, self.goals[0], False)
        posteriorProbabilityOfWorldB = self.oneColourBeliefVector2(self.traj2, self.nonUniformPriors, worldB, self.goals[0], False)
        self.assertTrue(posteriorProbabilityOfWorldA<posteriorProbabilityOfWorldB)
        
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2}, True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0},True),[(0,1), (0,1), (0,2), (1,0), (1,2), (1,0), (2,2), (0,-1), (2,1)]), (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0}, True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2},True),[(0,1), (1,0), (1,1), (1,0), (2,1)]))
    @unpack
    def test_actionInterpretationPosterior_FirstEnvGreaterProbability_oneColourStateSpace1(self, world1, world2, traj):
        #set up the correct combination of worlds trajectories and environments
        posteriorProbabilityOfWorld1 = self.oneColourBeliefVector1(traj, self.oneColourUniformPriors, world1, self.goals[0], False)
        posteriorProbabilityOfWorld2 = self.oneColourBeliefVector1(traj, self.oneColourUniformPriors, world2, self.goals[0], False)
        self.assertTrue(posteriorProbabilityOfWorld1>posteriorProbabilityOfWorld2)
    
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0}, True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2},True),[(0,1), (0,1), (0,2), (1,0), (1,2), (1,0), (2,2), (0,-1), (2,1)]))
    @unpack
    def test_actionInterpretationPosterior_FirstEnvGreaterProbability_oneColourStateSpace2(self, world1, world2, traj):
        #set up the correct combination of worlds trajectories and environments
        posteriorProbabilityOfWorld1 = self.oneColourBeliefVector2(traj, self.oneColourUniformPriors, world1, self.goals[0], False)
        posteriorProbabilityOfWorld2 = self.oneColourBeliefVector2(traj, self.oneColourUniformPriors, world2, self.goals[0], False)
        self.assertTrue(posteriorProbabilityOfWorld1>posteriorProbabilityOfWorld2)
        
    """
    Checking for the following cases:
        trajI -> B most likely
        trajII -> C most likely
        trajIII -> D most likely
        trajIV -> B most likely
    
    """
    @data( (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},True), [(0,1), (0,1), (0,2), (1,0), (1,2), (1,0), (2,2), (0,-1), (2,1)]), 
          #(sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0, 'blue':0}, True), [(0,1), (1,0), (1,1), (1,0), (2,1)]), 
          (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0, 'blue':-2}, True), [(0,1), (1,0), (1,1), (1,0), (2,1)]), 
          #(sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},True), [(0,1), (0,-1), (0,0), (1,0), (1,0), (1,0), (2,0), (0,1), (2,1)]), 
          (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':-2},True), [(0,1), (0,-1), (0,0), (1,0), (1,0), (1,0), (2,0), (0,1), (2,1)]), 
          (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},True), [(0,1), (0,1), (0,2)]) )
    @unpack
    def test_actionInterpretation_mostLikelyEnv_twoColourStateSpace(self, world, traj):
        belief = self.twoColourBeliefVector(traj, self.twoColourUniformPriors)
        mostLikelyEnv = max(belief, key= lambda x: belief[x]) 
        expectedMostLikelyEnv = (world, self.goals[0])
        self.assertEqual(mostLikelyEnv, expectedMostLikelyEnv)
    
    """
    Checking for the following cases to be equal:
        trajII -> A and C (equally and also most likely)
        trajIII -> B and D (equally and also most likely) FAILS
    
    """
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0, 'blue':0}, True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0, 'blue':-2}, True), [(0,1), (1,0), (1,1), (1,0), (2,1)]))
          #(sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':-2},True), [(0,1), (0,-1), (0,0), (1,0), (1,0), (1,0), (2,0), (0,1), (2,1)]) )
    @unpack
    def test_actionInterpretation_equallyLikelyEnvs_twoColourStateSpace(self, world1, world2, traj):
        probabilityEnv1 = self.twoColourBeliefVector(traj, self.twoColourUniformPriors, world1, self.goals[0], False)
        probabilityEnv2 = self.twoColourBeliefVector(traj, self.twoColourUniformPriors, world2, self.goals[0], False)
        self.assertAlmostEqual(probabilityEnv1, probabilityEnv2)
    
    """
    Checking for the following cases:
        trajII -> A>B 
        trajIII -> D>B 
    
    """
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': 0, 'blue':0}, True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0}, True), [(0,1), (1,0), (1,1), (1,0), (2,1)]),
          (sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':-2},True), sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0},True), [(0,1), (0,-1), (0,0), (1,0), (1,0), (1,0), (2,0), (0,1), (2,1)]) )
    @unpack
    def test_actionInterpretation_FirstGreaterThanTwo_twoColourStateSpace(self, world1, world2, traj):
        probabilityEnv1 = self.twoColourBeliefVector(traj, self.twoColourUniformPriors, world1, self.goals[0], False)
        probabilityEnv2 = self.twoColourBeliefVector(traj, self.twoColourUniformPriors, world2, self.goals[0], False)
        self.assertTrue(probabilityEnv1>probabilityEnv2)
    
    #Checking for Probability of B being the same in trajectory 1 and trajectory 4
    @data((sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0}, True), [(0,1), (0,1), (0,2), (1,0), (1,2), (1,0), (2,2), (0,-1), (2,1)], [(0,1), (0,1), (0,2)]))
    @unpack      
    def test_actionInterpretation_equallyLikelyTrajectories_twoColourStateSpace(self, world, traj1, traj2):
        probabilitytraj1 = self.twoColourBeliefVector(traj1, self.twoColourUniformPriors, world, self.goals[0], False)
        probabilitytraj2 = self.twoColourBeliefVector(traj2, self.twoColourUniformPriors, world, self.goals[0], False)
        probabilityTraj1Rounded = round(probabilitytraj1, 1)
        probabilityTraj2Rounded = round(probabilitytraj2, 1)
        self.assertAlmostEqual(probabilityTraj1Rounded,probabilityTraj2Rounded)
    
    #Trajectory is taken to be the first step in trajectoryIII and the worlds chosen are B and D
    @data( ([(0,1), (0,-1), (0,0)], sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':0}, True), 0.4974883),
           ([(0,1), (0,-1), (0,0)], sup.HashableWorld({'white': 0, 'yellow': 10, 'orange': -2, 'blue':-2}, True), 0.9924535) )
    @unpack
    def test_actionInterpretation_testGetLikelihood_twoColourStateSpace(self, traj, world, expectedLikelihood):
        likelihood =  self.twoColourBeliefVector.getLikelihood(traj, (world,self.goals[0]))
        likelihoodRounded = round(likelihood, 5)
        expectedLikelihood = round(expectedLikelihood, 5)
        self.assertEqual(likelihoodRounded, expectedLikelihood)
        
    def tearDown(self):
        pass


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.458s

OK
